<font size="10">Best place to eat pizza in Texas.</font>

In [5]:
!pip install folium 
!pip install wikipedia

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 109 kB 8.9 MB/s eta 0:00:01
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11686 sha256=cce1fc4e3902ad236c39fa86610a01ba1bcfcec768aea289c82999a194222337
  Stored in directory: /home/jovyan/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [9]:
CLIENT_ID = 'U5Q1AO1MY55VKQDTPNCTMASWN5GHZMD2FNSY5BNACIADYKHA' # your Foursquare ID
CLIENT_SECRET = 'XOF3EVKV4B5OFPURB3OOTLHW2SQJJHHVQBHBXLMK4R5LQ1T4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: U5Q1AO1MY55VKQDTPNCTMASWN5GHZMD2FNSY5BNACIADYKHA


In [70]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["Dallas, TX", 'San Antonio, TX', 'Austin, TX', 'Houston, TX', 'College Station, TX']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [71]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [72]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in Dallas, TX =  188
Showing Top 100
Total number of pizza places in San Antonio, TX =  170
Showing Top 100
Total number of pizza places in Austin, TX =  168
Showing Top 100
Total number of pizza places in Houston, TX =  198
Showing Top 100
Total number of pizza places in College Station, TX =  64
Showing Top 100


<font size="5">Lets us show where the top pizza place are in these five cities in Texas.</font>

In [73]:
maps[cities[0]]

In [74]:
maps[cities[1]]

In [75]:
maps[cities[2]]

In [76]:
maps[cities[3]]

In [77]:
maps[cities[4]]

<font size="5">Next we find the the mean location of the pizza places in every city.</font> \
<font size="8">Best place to find a hotel if you like Pizza</font>

In [78]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Dallas, TX
Mean Distance from Mean coordinates
0.10358934209164257
San Antonio, TX
Mean Distance from Mean coordinates
0.11006413355250629
Austin, TX
Mean Distance from Mean coordinates
0.09089413770247289
Houston, TX
Mean Distance from Mean coordinates
0.1295251710173956
College Station, TX
Mean Distance from Mean coordinates
0.030806298016328514


In [79]:
maps[cities[0]]

In [80]:
maps[cities[1]]

In [81]:
maps[cities[2]]

In [82]:
maps[cities[3]]

In [83]:
maps[cities[4]]